# Populate the AI knowledge base



In [1]:
!pip install -qU pinecone-client langchain_community cohere PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import cohere
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
import os
import getpass
from langchain.schema import Document
from langchain.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone as Pinecone_Langchain
import random

In [3]:
index = 'dispatch-ai'
os.environ['PINECONE_API_KEY'] = pinecone_secret_key = getpass.getpass('Enter Pinecone secret key:')
cohere_secret_key = getpass.getpass('Enter Cohere secret key:')

Enter Pinecone secret key:··········
Enter Cohere secret key:··········


In [4]:
pc = Pinecone(api_key=pinecone_secret_key)
if index not in pc.list_indexes().names():
  pc.create_index(
    name = index,
    dimension = 4096,
    metric = 'cosine',
    spec=ServerlessSpec(
      cloud="aws",
      region="us-west-2"
      )
  )
pc.describe_index(index)

{'dimension': 4096,
 'host': 'dispatch-ai-r4usyr2.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'dispatch-ai',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [5]:
# Each document will have a size of 500 or less
character_text_splitter = CharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex=False,
)
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex=False,
)

In [6]:
import re

# A function to transform the Google sheet url to a downloadable CSV url
def convert_google_sheet_url(url):
    """

    Args:
    - url (str): The Google Sheets URL to convert.

    Returns:
    - str: The converted URL pointing to a downloadable CSV.
    """

    # Regular expression pattern to match Google Sheets URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'
    # Replacement function to convert the URL
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'
    new_url = re.sub(pattern, replacement, url)
    return new_url

In [ ]:
# Fill the vector database with curated articles about different emergency types
sheet_url = 'https://docs.google.com/spreadsheets/d/1smCE64VgagNaTDAMeGqSRX61z70orrkd10kzvDjRf0c/edit?usp=sharing'
df = pd.read_csv(convert_google_sheet_url(sheet_url))
df.head()

,Type of emergency,Answer,Reference,Link
0,Cardiac Arrest,Cardiac arrest is when the heart stops pumping...,MSD manual,https://www.msdmanuals.com/home/heart-and-bloo...
1,Cardiac Arrest,First-Aid Treatment for Cardiac Arrest\nThe cr...,MSD manual,https://www.msdmanuals.com/home/heart-and-bloo...
2,Cardiac Arrest,The crucial links in the chain of survival inc...,MSD manual,https://www.msdmanuals.com/home/heart-and-bloo...
3,Cardiac Arrest,Automated External Defibrillator: Jump-Startin...,MSD manual,https://www.msdmanuals.com/home/heart-and-bloo...
4,Cardiac Arrest,Skill in CPR is best obtained through a traini...,MSD manual,https://www.msdmanuals.com/home/heart-and-bloo...


In [ ]:
documents = []
for _, row in df.iterrows():
    chuncks = character_text_splitter.split_text(f"{row['Answer']}")
    for chunck in chuncks:
      documents.append(Document(
          page_content = chunck,
          metadata = {"Reference": f"{row['Reference']}", "Link": f"{row['Link']}"}))

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_secret_key, user_agent='dispatch-ai')
vector_store = Pinecone_Langchain.from_documents(documents, embeddings, index_name=index)

In [ ]:
retriever = vector_store.as_retriever()
matched_docs = retriever.get_relevant_documents('What is Cardiac Arrest?')
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)
    print(d.metadata)


## Document 0

Cardiac arrest is when the heart stops pumping blood and oxygen to the brain and other organs and tissues. Sometimes a person can be revived after cardiac arrest, particularly if treatment is started immediately. However, the more time that passes without oxygen-containing blood being pumped to the brain, the less likely it is that the person can be revived, and, if revived, the more likely it is that the person will have brain damage.
{'Link': 'https://www.msdmanuals.com/home/heart-and-blood-vessel-disorders/cardiac-arrest-and-cpr/cardiac-arrest-and-cpr#Standard-CPR_v79738929', 'Reference': 'MSD manual'}

## Document 1

Cardiac arrest can be caused by anything that makes the heart stop beating. One common cause, especially in adults, is an abnormal heart rhythm (arrhythmia). Another possible cause is stopping breathing, such as when a person drowns or has a severe lung infection or severe asthma attack.

A person in cardiac arrest lies motionless and does not respond t

In [ ]:
# Adding some useful books to the knowledge base
doc_reader = PdfReader('/content/Emergency Call Management - People at risk.pdf')

In [ ]:
raw_text = ''
for _, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
chunks = recursive_text_splitter.split_text(raw_text)

In [ ]:
docs = []
for chunk in chunks:
  docs.append(Document(
        page_content=chunk,
        metadata={"Reference": "Emergency call management: People at risk", "Link": "https://nfcc.org.uk/wp-content/uploads/2023/11/EmergencycallmanagementpeopleatriskConsultationdoc.pdf"},
  ))

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_secret_key, user_agent='dispatch-ai')
vector_store = Pinecone_Langchain.from_documents(documents, embeddings, index_name=index)

In [ ]:
doc_reader = PdfReader('/content/Emergency Medical Dispatch Procedure.pdf')

In [ ]:
raw_text = ''
for _, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
docs = []
for chunk in chunks:
  docs.append(Document(
        page_content=chunk,
        metadata={"Reference": "Emergency Medical Dispatch Procedure", "Link": "http://ma911.org/_wp/wp-content/uploads/2015/06/EMD-Procedures-March-08.pdf"},
  ))

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_secret_key, user_agent='dispatch-ai')
vector_store = Pinecone_Langchain.from_documents(documents, embeddings, index_name=index)

In [7]:
# Fill the vector database with simulations
sheet_url = 'https://docs.google.com/spreadsheets/d/1J8gwsPwp8gc2cue-S1eu9I_8pcTQ2oezXnE6RZWGhMg/edit?usp=sharing'
df = pd.read_csv(convert_google_sheet_url(sheet_url))
df.head()

,Type of emergency,scenario,Title,protocol steps,conversation,reference,link
0,Cardiac Arrest,A family member finds an elderly person uncon...,Elderly Cardiac Arrest Rescue,Protocol steps:\nCheck Responsiveness: Shake t...,"Dispatcher: ""911, what's your emergency?""\nCal...",Emergency Medical Dispatch Guide Cards,https://ia903008.us.archive.org/view_archive.p...
1,Cardiac Arrest,"A person collapses at a shopping mall, and a b...",Cardiac Arrest at Shopping Mall,Check Responsiveness: Ensure the person is unr...,"Dispatcher: ""911, what's your emergency?""\nCal...",Emergency Medical Dispatch Guide Cards,https://ia903008.us.archive.org/view_archive.p...
2,Cardiac Arrest,"A gym member collapses during a workout, and t...",Gym Member Cardiac Arrest,Check Responsiveness: Assess if the person is ...,"Dispatcher: ""911, what’s your emergency?""\nCal...",Emergency Medical Dispatch Guide Cards,https://ia903008.us.archive.org/view_archive.p...
3,Cardiac Arrest,"A student collapses in the classroom, and the...",Cardiac Arrest: Student Collapse in Classroom,Check Responsiveness: Verify if the student is...,"Dispatcher: ""911, what’s your emergency?""\nCal...",Emergency Medical Dispatch Guide Cards,https://ia903008.us.archive.org/view_archive.p...
4,Choking,An adult is experiencing a partial airway obst...,Partial Airway Obstruction: Distressed Adult C...,Confirm that the person is choking.\nEncourage...,"Dispatcher: ""911, what is your emergency?""\nCa...",Emergency Medical Dispatch Guide Cards,https://ia903008.us.archive.org/view_archive.p...


In [ ]:
documents = []
for _, row in df.iterrows():
    chuncks = character_text_splitter.split_text(f"""Scenario: {row['scenario']}
                                                 Protocol: {row['protocol steps']}
                                                 conversation: {row['conversation']}""")
    for chunck in chuncks:
      documents.append(Document(
          page_content = chunck,
          metadata = {"Emergency Type": f"{row['Type of emergency']}", "Title": f"{row['Title']}", "Type": "Scenario", "Reference": f"{row['reference']}", "Link": f"{row['link']}"}))

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_secret_key, user_agent='dispatch-ai')
vector_store = Pinecone_Langchain.from_documents(documents, embeddings, index_name=index)

In [8]:
# Add protocols for the Q&A feature
documents = []
for _, row in df.iterrows():
    chuncks = character_text_splitter.split_text(f"""This is a protocol for this scenario: {row['scenario']}
                                                 The steps: {row['protocol steps']}""")
    for chunck in chuncks:
      documents.append(Document(
          page_content = chunck,
          metadata = {"Reference": f"{row['reference']}", "Link": f"{row['link']}"}))

In [9]:
embeddings = CohereEmbeddings(cohere_api_key= cohere_secret_key, user_agent='dispatch-ai')
vector_store = Pinecone_Langchain.from_documents(documents, embeddings, index_name=index)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(
